<a href="https://colab.research.google.com/github/luiseufrasio/safe-route-ai/blob/main/safe_route.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Instalando o SDK do Google

In [59]:
!pip install -q -U google-generativeai

###Configurando a API KEY do Google Gemini

In [60]:
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get("SECRET_KEY"))

###Criando a lista de mapas e o método para upload a partir do caminho no Colab

In [61]:
from pathlib import Path
import hashlib

uploaded_maps = []
def upload_if_needed(pathname: str) -> list[str]:
  path = Path(pathname)
  hash_id = hashlib.sha256(path.read_bytes()).hexdigest()
  try:
    existing_file = genai.get_file(name=hash_id)
    return [existing_file]
  except:
    pass
  uploaded_maps.append(genai.upload_file(path=path, display_name=hash_id))
  return [uploaded_maps[-1]]

#Carregando o mapa topográfico do Estado do RS
##### Fonte: https://pt-br.topographic-map.com/map-pb651/Rio-Grande-do-Sul/

In [87]:
rs_input_parts = ["input: ", *upload_if_needed("/content/RS-Metropolitana.png"), "Analise esse mapa TOPOGRÁFICO de cores do Rio Grande de Sul, tente identificar as cidades que estão nele e me dê sua altitude aproximada com base nas cores. Mostre o resultado ordenado pela altitude encontrada."]

#Setup do Modelo

In [79]:
# Set up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

#Executando a pesquisa e mostrando a resposta do modelo

In [94]:
response1 = model.generate_content(rs_input_parts)

In [95]:
from IPython.display import Markdown

display(Markdown(response1.text))

## Análise do Mapa Topográfico do Rio Grande do Sul:

Com base no mapa topográfico fornecido, podemos identificar as seguintes cidades e suas altitudes aproximadas, ordenadas da maior para a menor altitude:

**Cidade** | **Altitude (m)** | **Cor no Mapa**
------- | -------- | --------
Torres | 1049 | Vermelho Escuro
São Francisco de Paula | ~950 | Vermelho
Canela | ~850 | Laranja Avermelhado
Três Coroas | ~750 | Laranja
Igrejinha | ~650 | Amarelo Alaranjado
Taquara | ~550 | Amarelo Esverdeado
Rolante | ~450 | Verde Claro
Osório | ~350 | Verde Azulado
Santo Antônio da Patrulha | ~250 | Azul Esverdeado
Tramandaí | ~150 | Azul Claro
Cidreira | ~100 | Azul Claro
Palmares do Sul | ~50 | Azul Muito Claro
Balneário Pinhal | ~10 | Azul Muito Claro
 
**Observações:**
* As altitudes são aproximadas, baseadas na escala de cores do mapa.
* Nem todas as cidades são visíveis ou possuem nomes no mapa.
* As cores do mapa representam faixas de altitude, e a transição entre as cores pode não ser perfeitamente precisa.

Espero que esta análise seja útil! 😊 


#Perguntando a Pessoa qual cidade ela está

In [72]:
lista_cidades = ['Canoas', 'Lajeado', 'Pelotas', 'Porto Alegre', 'Rio Grande', 'São Leopoldo']
cidade = input("Em qual dessas cidades você está: [" + ", ".join(lista_cidades) + "] ?\t")

while cidade not in lista_cidades:
  print("A cidade escolhida não está na lista. Escolha novamente...")
  cidade = input("[" + ", ".join(lista_cidades) + "] \t")

Em qual dessas cidades você está: [Canoas, Lajeado, Pelotas, Porto Alegre, Rio Grande, São Leopoldo] ?	Rio Grande


#Saindo da cidade escolhida para cidades com altitude acima dos 100m

In [97]:
destinos = ['Torres', 'São Francisco de Paula', 'Canela', 'Três Coroas', 'Igrejinha', 'Taquara', 'Rolante', 'Osório', 'Santo Antônio da Patrulha', 'Tramandaí']
cidade_input = "Trace rotas partindo da cidade " + cidade + " no estado do Rio Grande do Sul para 3 cidades nessa lista: [" + ", ".join(destinos) + "]. Evite caminhos que possam passar por pontes."

response2 = model.generate_content(cidade_input)

display(Markdown(response2.text))

## Rotas partindo de Rio Grande - RS, evitando pontes:

É importante notar que, devido à geografia da região, evitar pontes completamente pode ser extremamente difícil e, em alguns casos, impossível. As rotas a seguir minimizam o uso de pontes, mas algumas podem ser inevitáveis.

**1. Rio Grande - Torres:**

* Esta rota é direta pela RS-040, sem necessidade de pontes.

**2. Rio Grande - São Francisco de Paula/Canela/Três Coroas:**

* Infelizmente, para chegar a essas cidades, a travessia da Lagoa dos Patos por ponte é **inevitável**.
* A rota mais comum utiliza a Ponte do Guaíba (BR-290) e segue pela BR-116 até as cidades desejadas.

**3. Rio Grande - Igrejinha/Taquara/Rolante:**

*  Assim como as cidades acima, a travessia da Lagoa dos Patos por ponte é **inevitável** para chegar a essas cidades.
*  A rota mais comum utiliza a Ponte do Guaíba (BR-290) e segue pela BR-116 e, posteriormente, RS-239 para acessar as cidades.

**4. Rio Grande - Osório:**

* Esta rota é direta pela RS-040, sem necessidade de pontes.

**5. Rio Grande - Santo Antônio da Patrulha:**

*  Atravessar a Lagoa dos Patos por ponte é **inevitável**.
* A rota mais comum utiliza a Ponte do Guaíba (BR-290) e segue pela BR-290 até Santo Antônio da Patrulha.

**6. Rio Grande - Tramandaí:**

* É possível evitar a Ponte do Guaíba utilizando a Rota do Sol (RS-486), que contorna a Lagoa dos Patos pelo litoral. 
* Esta rota pode ter trechos com pontes menores, mas evita a grande ponte sobre a Lagoa dos Patos. 

**Observações:**

* É importante verificar as condições das estradas antes de viajar, especialmente na Rota do Sol, que pode ter trechos em obras ou com restrições de tráfego.
* Utilizar aplicativos de navegação com a opção "evitar pedágios" pode auxiliar na identificação de rotas alternativas que minimizem a passagem por pontes, mas lembre-se de que a precisão destes aplicativos pode variar.

Espero que essas informações sejam úteis para planejar suas viagens! 


#Chat Final para mais dúvidas

In [90]:
chat = model.start_chat(history=[rs_input_parts, response1, cidade_input, response2])
sair = "[digite 'sair' para finalizar a conversa]"
prompt = input("Ajudo em algo mais? ", sair)

while prompt != 'sair':
  response = chat.send_message(prompt)
  print("Modelo: ")
  display(Markdown(response.text))
  print("[digite 'sair' para finalizar a conversa]\n")
  prompt = input("Você: ")

Modelo: 


Infelizmente, não consigo analisar mapas visuais, identificar cidades ou traçar rotas de carro. Sou um modelo de linguagem e não possuo a capacidade de processar informações visuais como um mapa. 

Para obter rotas de carro e informações de elevação, você pode usar aplicativos de navegação como:

* **Google Maps:** Permite traçar rotas, visualizar o terreno e obter informações de elevação.
* **Waze:** Oferece navegação em tempo real e pode ajudar a evitar pontes.

Ao usar esses aplicativos, você pode inserir seu destino e especificar que deseja evitar pontes nas configurações de rota. 


[digite 'sair' para finalizar a conversa]

Você: sair
